# Sentiment Analysis of Stock/Market News

For the purpose of classifying news as positive or negative (or neutral) we will use VADER.

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media

In [56]:
# Import libraries
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import psycopg2
import psycopg2.extras


In [47]:
# Import stock news data

stock_news = pd.read_csv (r'C:\Users\imaia\Desktop\Stats170A\Capstone\stock_news_data.csv')


In [48]:
stock_news.head()

,headline,source,summary,url,date
0,Dow Futures Soar as New York Begins Winning th...,ccn.com,NaN,https://www.ccn.com/dow-futures-soar-as-new-yo...,2020-04-06T05:41:20+00:00
1,Apple Snaps Up AI Startup Inductiv As Analysts...,smarteranalyst.com,NaN,https://www.smarteranalyst.com/yahoo/apple-sna...,2020-05-28T02:38:10+00:00
2,Gates Foundation Buys Up Amazon Apple Twitter ...,smarteranalyst.com,NaN,https://www.smarteranalyst.com/yahoo/gates-fou...,2020-05-25T11:02:06+00:00
3,Apple China Sales On Recovery Path In April iP...,smarteranalyst.com,NaN,https://www.smarteranalyst.com/yahoo/apple-chi...,2020-05-25T03:13:23+00:00
4,Intel Taiwan Semiconductor Said to Be in Talks...,smarteranalyst.com,NaN,http://www.smarteranalyst.com/uncategorized/in...,2020-05-13T05:13:13+00:00


In [49]:
stock_news.shape

(8267, 5)

In [50]:
# Sentiment Analysis of Stock News
analyzer = SentimentIntensityAnalyzer()

scores = []

for i in stock_news['headline']:
    
    init_scores = analyzer.polarity_scores(str(i))
    scores.append(init_scores['compound'])

df_scores = pd.DataFrame(scores)

stock_news = stock_news.join(df_scores, rsuffix='_right')


In [51]:
stock_news.head()

,headline,source,summary,url,date,0
0,Dow Futures Soar as New York Begins Winning th...,ccn.com,NaN,https://www.ccn.com/dow-futures-soar-as-new-yo...,2020-04-06T05:41:20+00:00,-0.1280
1,Apple Snaps Up AI Startup Inductiv As Analysts...,smarteranalyst.com,NaN,https://www.smarteranalyst.com/yahoo/apple-sna...,2020-05-28T02:38:10+00:00,0.4019
2,Gates Foundation Buys Up Amazon Apple Twitter ...,smarteranalyst.com,NaN,https://www.smarteranalyst.com/yahoo/gates-fou...,2020-05-25T11:02:06+00:00,0.1779
3,Apple China Sales On Recovery Path In April iP...,smarteranalyst.com,NaN,https://www.smarteranalyst.com/yahoo/apple-chi...,2020-05-25T03:13:23+00:00,0.0000
4,Intel Taiwan Semiconductor Said to Be in Talks...,smarteranalyst.com,NaN,http://www.smarteranalyst.com/uncategorized/in...,2020-05-13T05:13:13+00:00,0.0000


In [52]:
stock_news = stock_news.drop(['summary'], axis = 1)
stock_news.columns = ['headline', 'source', 'url', 'date', 'score']

In [53]:
stock_news['date'] = stock_news['date'].astype(str).str[0:10]

In [54]:
stock_news.head()

,headline,source,url,date,score
0,Dow Futures Soar as New York Begins Winning th...,ccn.com,https://www.ccn.com/dow-futures-soar-as-new-yo...,2020-04-06,-0.1280
1,Apple Snaps Up AI Startup Inductiv As Analysts...,smarteranalyst.com,https://www.smarteranalyst.com/yahoo/apple-sna...,2020-05-28,0.4019
2,Gates Foundation Buys Up Amazon Apple Twitter ...,smarteranalyst.com,https://www.smarteranalyst.com/yahoo/gates-fou...,2020-05-25,0.1779
3,Apple China Sales On Recovery Path In April iP...,smarteranalyst.com,https://www.smarteranalyst.com/yahoo/apple-chi...,2020-05-25,0.0000
4,Intel Taiwan Semiconductor Said to Be in Talks...,smarteranalyst.com,http://www.smarteranalyst.com/uncategorized/in...,2020-05-13,0.0000


In [60]:
# Store stock news data in PostgreSQL database
conn = None

try:
    conn = psycopg2.connect(host="localhost", database="postgres", port=5432, user="postgres", password="Aki12345+")

    command = "INSERT INTO stock_news (headline, source, url, date, score) \
    VALUES (%s, %s, %s, %s, %s);"

    cur = conn.cursor()
    for index, row in stock_news.iterrows():
        cur.execute(command, (str(row['headline']),str(row['source']),
                              str(row['url']), str(row['date']), float(row['score'])))

except (Exception, psycopg2.DatabaseError):
    print("Could not connect to database")

finally:
    if conn is not None:
        conn.commit()
        conn.close()
        cur.close()